In [1]:
import sys
from kaitoupao import *

/Users/minkexiu/Documents/GitHub/Tianchi_AIFinanceTimeSeries/kaitoupao.py:280: SyntaxWarning: invalid escape sequence '\d'
  *[int(x) for x in re.findall("\d+", zh_date_str)]


storage dir: /Users/minkexiu/Downloads/GitHub/Tianchi_AIFinanceTimeSeries
code dir: /Users/minkexiu/Documents/GitHub/Tianchi_AIFinanceTimeSeries 

09 16 14
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：1 本卦下：8 变爻：2


,天地否,风山渐,天水松
上卦,☰乾金,☴巽木,☰乾金
下卦,☷坤土,☶艮土,☵坎水


03 12 9 申时
先天八卦数: 1乾, 2兑, 3离, 4震, 5巽, 6坎, 7艮, 8坤
本卦上：3 本卦下：4 变爻：3


,火雷噬嗑,水山蹇,火火离
上卦,☲离火,☵坎水,☲离火
下卦,☳震木,☶艮土,☲离火


基础的数据分析。

大致设定一些数据分析目标：
* 到底有多少个product_id。--》139个。
* 每个product_id序列都有多长。--〉最长448，最短也有30+。
* 每个product不是有三个数值嘛，看看都是什么样的分布。这个就是有一个直观理解。可能没什么直接的帮助。
* 是不是所有的apply_amt和redeem_amt的差都等于net_in_amt。

In [2]:
df_pred = load_data_from_originalData("predict_table")
df_train = load_data_from_originalData("product_info_simple_final_train")

pd.read_csv("/Users/minkexiu/Downloads/GitHub/Tianchi_AIFinanceTimeSeries/originalData/predict_table.csv", quoting=3, lineterminator="\n")
pd.read_csv("/Users/minkexiu/Downloads/GitHub/Tianchi_AIFinanceTimeSeries/originalData/product_info_simple_final_train.csv", quoting=3, lineterminator="\n")


In [10]:
df_pred.shape, df_train.shape

((1390, 5), (43399, 12))

In [4]:
df_train.product_pid.value_counts()
## 训练数据里的productid，长度参差不齐。

product_pid
product12     448
product35     448
product49     448
product48     448
product46     448
             ... 
product69      15
product5       10
product154      7
product74       5
product77       4
Name: count, Length: 157, dtype: int64

In [5]:
df_pred.product_pid.value_counts()
## 看来所有的pred里面的产品，都只有10个数据。

product_pid
product1      10
product56     10
product50     10
product51     10
product52     10
              ..
product138    10
product137    10
product136    10
product135    10
product99     10
Name: count, Length: 139, dtype: int64

In [6]:
set(df_pred.product_pid) - set(df_train.product_pid)

set()

In [7]:
set(df_train.product_pid) - set(df_pred.product_pid)

{'product10',
 'product11',
 'product127',
 'product139',
 'product154',
 'product18',
 'product21',
 'product22',
 'product23',
 'product25',
 'product36',
 'product4',
 'product44',
 'product5',
 'product69',
 'product7',
 'product74',
 'product77'}

In [8]:
## 说明部分的product是没有出现在pred里面的。

In [ ]:
## 那我们继续看看，出现在数据里面的product都有啥。

In [11]:
df_train = df_train[
    df_train.product_pid.isin(
        df_pred.product_pid.to_list()
    )
].reset_index(drop=True)

In [13]:
df_train.product_pid.value_counts()
## 说明，最少的序列长度都有31. 
## 最长的有448.
## 应该是够预测了的。

product_pid
product29     448
product37     448
product15     448
product49     448
product48     448
             ... 
product156     86
product157     84
product151     72
product152     38
product121     31
Name: count, Length: 139, dtype: int64

In [22]:
(
    (df_train["apply_amt"] - df_train["redeem_amt"]) - (df_train["net_in_amt"]) < 1e-6
).sum(), df_train.shape[0]
## 靠背哦，其实说白了，apply_amt跟redeem_amt之间的差就是net_in_amt。误差很小的，基本就不要管它了罢。

(42848, 42848)

In [17]:
## 看来就是不完全一样的。
## 【TODO】回头我们应该看看，这里不同的，都是什么原因导致的。怎么会是这种情况呢。。。

In [18]:
df_train[
    (df_train["apply_amt"] - df_train["redeem_amt"]) (df_train["net_in_amt"])
]

,product_pid,transaction_date,apply_amt,redeem_amt,net_in_amt,uv_fundown,uv_stableown,uv_fundopt,uv_fundmarket,uv_termmarket,during_days,total_net_value
0,product1,20210104,0.001642,0.000851,0.000791,7.918270e+05,0.0,8203723.0,0.0,0.0,0.0,NaN
2,product1,20210106,0.002941,0.001342,0.001599,7.590310e+05,0.0,6924679.0,0.0,0.0,0.0,NaN
8,product1,20210114,0.000102,0.000339,-0.000237,4.310710e+05,0.0,2562811.0,37519.0,0.0,0.0,NaN
12,product1,20210120,0.004303,0.010718,-0.006416,5.950510e+05,0.0,5711227.0,0.0,0.0,0.0,NaN
13,product1,20210121,0.000119,0.000312,-0.000193,2.014990e+05,0.0,1972483.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
42840,product99,20221031,11.465688,12.210197,-0.744509,1.637050e+09,713481703.0,423302695.0,4956919.0,591316603.0,90.0,4.354314
42841,product99,20221101,4.002042,12.474778,-8.472736,1.155933e+09,416251555.0,388637323.0,1677319.0,319109803.0,90.0,4.353897
42842,product99,20221102,6.106381,6.216262,-0.109881,1.493108e+09,455311591.0,493158175.0,1775707.0,361810195.0,90.0,4.354314
42843,product99,20221103,6.420888,7.034544,-0.613657,1.343788e+09,428517259.0,448850779.0,1349359.0,363745159.0,90.0,4.354314


In [21]:
1.967026 - 21.100487

-19.133461